In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Understanding


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd

In [ ]:
df = pd.read_csv('../input/us-airbnb-open-data/AB_US_2020.csv')

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

Will exclude the rows having Null values.

In [ ]:
df1=df.dropna()
df1.info()

In [ ]:
unique_columns1= [df1[col].nunique() for col in df1]
count=pd.DataFrame({"Unique" : unique_columns1})
count['Column']=list(df1.columns)
count

In [ ]:
df1.name = df1.name.astype(str)
df1.name=df1.name.str.lower()
df1.name=df1.name.replace(['-','@','$','&','!'],'',regex=True)

To replace special characters from name column

In [ ]:
from wordcloud import WordCloud,STOPWORDS
comment_words = '' 
stopwords = set(STOPWORDS) 
for val in df1.name: 
    val = str(val) 
    tokens = val.split() 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower()       
    comment_words += " ".join(tokens)+" "  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)                        
plt.figure(figsize = (8, 12), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)   
plt.show() 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

Loading stop words to exclude them from the text in columns

In [ ]:
df1["unigrams"] = df1["name"].apply(nltk.word_tokenize)
df1["unigrams"] =df1["unigrams"].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:

rslt=pd.Series(np.concatenate([x for x in df1.unigrams])).value_counts()
rslt = pd.DataFrame({'ngrams': list(rslt.keys()),
                   'count': list(rslt[:])})
rslt=rslt[rslt.ngrams != '.' ]
rslt=rslt.head(20)
rslt1=rslt['ngrams'][1:20]
rslt1

20 most frequently observed words in Name column

# Data Processing

In [ ]:

def inner(text):    
    for word in text.split(' '):
        for c in rslt1:
            if word==c:
               return c
    return 'Null'       
c1=[]
for text in df1['name']:
    c1.append(inner(text))
df1['Name1']=c1
df1.info()

In [ ]:
df2=df1[['Name1','neighbourhood_group','room_type',
        'city','price']]

plt.hist(df2['price'], bins =20)
plt.show()

considering those cases with price less than $ 1000

In [ ]:
df2=df2[(df2['price']<1000)  ]

In [ ]:

names = list(df2.columns)
fig, axs = plt.subplots(4, 1,figsize=(15,15))
plt.subplots_adjust(left = 0.25, right = 0.9, bottom = 0.1, top = 1.5,
                        wspace = 0.2, hspace = 1.2)

for column_name in names[:-1]: 
    ax = axs[names.index(column_name)]
    cd= df2[column_name].value_counts()[:5]
    cd1= pd.DataFrame({'label': list(cd.keys()),
                   'count': list(cd[:])})
    s=cd1['label']
    
    for s1 in s:
        # Subset to the airline
        subset = df2[df2[column_name] == s1]
        
        # Draw the density plot
        sns.distplot(subset['price'], hist = False, kde = True,
                     kde_kws = {'shade': True,'linewidth': 1},ax=ax,
                     label = s1).set(xlim=(0))
        ax.set_xlabel( ' Target Price')
        ax.set_title(column_name + ' density')
        ax.grid('on')

Density distribution for top 5 frequencies of Name1, neighbourhood group,room type and city against Price.

In [ ]:
df_group = df2.groupby('Name1').mean()  
df_group.sort_values(by=['price'], ascending=False) 


Comparison of distribution of price for top two mean price vs bottom two mean price based on text (name)

In [ ]:
names = ['Name1','price']


for column_name in names[:-1]: 
    cd=['ocean','beach','Null','cozy','room']
    cd1= pd.DataFrame(cd,columns=['label'])
    s=cd1['label']
    
    for s1 in s:
        # Subset to the airline
        subset = df2[df2[column_name] == s1]
        
        # Draw the density plot
        ax=sns.distplot(subset['price'], hist = False, kde=True,
                     kde_kws = {'shade': True,'linewidth': 2},
                     label = s1).set(xlim=(0))

In [ ]:
x1=[]
for c in df1.last_review:
    x1.append([c[:2],c[3:5],c[6:8]])
dt = pd.DataFrame(x1, columns=['day', 'month', 'year'],index=df1.index)
dt[['day', 'month', 'year']] = dt[['day', 'month', 'year']].astype(int)
dt.year=dt.year+2000

import datetime

dt['rev']= pd.to_datetime(dt.year*10000+dt.month*100+dt.day,format='%Y%m%d')
dt['rev']= dt['rev'].dt.date
dt['now']=datetime.date.today()
dt['dif']=dt.now-dt.rev
df1['days']=dt['dif']
df1['days']=(df1['days']/np.timedelta64(1, 'D')).astype(np.int64)

df1=df1[(df1['price']<1000)]
df1.info()

Number of days since last review. For cases where 

In [ ]:
df1 = df1.drop([  'id','host_id','latitude','longitude','last_review'], axis = 1)
corr = df1.corr()
print(corr)
plt.figure(figsize=(20,10))
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
dummy = pd.get_dummies(df1[['neighbourhood_group','room_type',
        'city','Name1']])

df11 = pd.concat([df1, dummy], axis = 1)
x = df11.drop(['Name1','neighbourhood_group','room_type',
        'city','price','name','host_name','neighbourhood','unigrams'], axis = 1)
x= (x-np.min(x)) / (np.max(x)-np.min(x))
y = df11['price']
y= (y-np.min(y)) / (np.max(y)-np.min(y))
x.info()

# Linear Regression

In [ ]:
                   
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2 , random_state = 21)


In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)
reg.score(x_train, y_train)
y1=reg.predict(x_test)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y1)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, y1)
print("R2:", r2)

In [ ]:
plt.scatter(y_test, y1)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
z = np.polyfit(y_test, y1, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='red')
plt.show()


# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
model=regressor.fit(x, y)  
y1 = model.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, y1)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, y1)
print("R2:", r2)

In [ ]:
plt.scatter(y_test, y1)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
z = np.polyfit(y_test, y1, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='red')
plt.show()


In [ ]:
n_nodes = []
max_depths = []
for ind_tree in model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')
